
# Data and computations for paper 

Lars G. Johnsen
National Library of Norway

In [1]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
mu.update('collocations')
from collocations import dist_coll_urn, urn_coll, calculate_midpoint, dist
import pandas as pd

Updated file `C:\Users\larsj\Documents\GitHub\newspapers_coll_conc\collocations.py`

In [2]:
def large_corpus_dist_coll(collword, urns = None, after= 10, before = 0, n = 300):
    colls_freq = []
    colls_dist = []
    coll = pd.DataFrame()
    for i in range(0, len(urns), n):
        a = urn_coll(collword, urns = urns[i:i + n], after= after, before = before)
        colls_freq.append(nb.frame(a['freq']))
        colls_dist.append(nb.frame(a['dist']))
    coll['freq'] = pd.concat(colls_freq, axis=1, sort=False).sum(axis=1)
    coll['dist'] = pd.concat(colls_dist, axis = 1, sort=False).mean(axis=1)
    coll['dist_score'] = round(dist(coll['dist'], calculate_midpoint(before, after), coll['freq']), 2)
    return coll

In [3]:
def large_corpus_coll(collword, urns = None, after= 5, before = 5, n = 300):
    colls = []
    for i in range(0, len(urns), n):
        colls.append(nb.urn_coll(collword, urns=urns[i:i + n], after= after, before = before))
    coll = pd.concat(colls, axis=1, sort=False).sum(axis=1)
    return pd.DataFrame(coll)

In [4]:
def make_coll_df(small, large, tot):
    """take two collocations, small and large, and a reference tot. The reference must av a column call tot as well"""
    coll = pd.DataFrame()
    coll['small'] = small['freq']
    coll['large'] = large['freq']
    #coll['ratio'] = coll['small']/coll['large']
    coll['srel'] = coll.small/coll.small.sum()
    coll['lrel'] = coll.large/coll.large.sum()
    coll['ratio'] = coll.srel/coll.lrel
    coll['adjusted'] = coll.small**0.01*coll.ratio
    coll['nb'] = coll.small/tot.tot
    return coll

### Reference corpus

The reference is the total counts from approximately 450 000 books from nb.no

# Corpus

The corpus for doing collocations is a sample of 800 books from fictional literature, dewey decimal code 813.

In [5]:
smd = 5
lmd = 10
collword = 'jeg'
corpus_size= 1000
period = (1980, 2000)

In [6]:
def filenames(smd = 5, lmd = 10, collword = 'motor', corpus_size= 1000, period = (1980, 2000)):
    return {
    'small_right': '_'.join([collword, str(smd), str(corpus_size), '-'.join([str(x) for x in period]), str(0), str(int(2*smd)), '.csv']),
    'large_right': '_'.join([collword, str(smd), str(corpus_size),  '-'.join([str(x) for x in period]), str(0), str(int(2*lmd)), '.csv']),
    'small_left' : '_'.join([collword, str(lmd), str(corpus_size),  '-'.join([str(x) for x in period]), str(int(2*smd)), str(0), '.csv']),
    'large_left' : '_'.join([collword, str(lmd), str(corpus_size),  '-'.join([str(x) for x in period]), str(int(2*lmd)), str(0), '.csv'])
        }

In [7]:
fnames = filenames(smd=smd, lmd=lmd, collword=collword, corpus_size=corpus_size, period=period)
fnames

{'small_right': 'jeg_5_1000_1980-2000_0_10_.csv',
 'large_right': 'jeg_5_1000_1980-2000_0_20_.csv',
 'small_left': 'jeg_10_1000_1980-2000_10_0_.csv',
 'large_left': 'jeg_10_1000_1980-2000_20_0_.csv'}

In [8]:
corpus = nb.book_corpus(words=[collword], period = period, limit = corpus_size)

In [9]:
corpus = nb.frame([x for x in nb.metadata(corpus) if x[8] == 'nob'], ['urn'])

In [10]:
print(len(corpus))
corpus.head()

858


,urn,1,2,3,4,5,6,7,8,9
0,2012082908107,"Iversen, Jan Roger",1987,"Bardu næringstjeneste, Tranøy næringstjeneste",prosjekt,Sentas,unknown,500f4d59867115232b50dba556dad2ef,nob,
1,2013092407101,"Seamands, David A.",1992,Fri fra vonde minner,,Rex,notfiction,0d47c31ea218f9ee85d133ec98da6454,nob,eng
2,2010120206007,"Santos, Sølvi dos",1998,Magiske Mallorca,stil og interiør på de spanske middelhavsøyene,Grøndahl Dreyer,notfiction,af23dbd7616f70bb39b30cfe05e66abf,nob,eng
3,2013091838081,,1986,Verdens dyr,,Den norske Bokklubben;[Oslo] : Den norske Bokk...,unknown,f16f8e3ad8781c7ce41712f2df5581a0,nob,
4,2007111301084,"Bernt, Jan Fridthjof",1989,Kommunalrett,,TANO,notfiction,6de318326a7dfe03b9787cbad4943477,nob,


Set up the distance parameters and collword, smd is small distance while lmd is large distance. These values are half of the actual window, and used to make a normalized score, call ascore. See below.

In [41]:
a1 = nb.frame(large_corpus_dist_coll(collword, urns=list(corpus.urn), after= int(2*smd), before = 0), 'freq')
print(1)
a2 = nb.frame(large_corpus_dist_coll(collword, urns=list(corpus.urn), after= int(2*lmd), before = 0), 'freq')  
print(2)
b1 = nb.frame(large_corpus_dist_coll(collword, urns=list(corpus.urn), after=0, before = int(2*smd)), 'freq')
print(3)
b2 = nb.frame(large_corpus_dist_coll(collword, urns=list(corpus.urn), after=0, before = int(2*lmd)), 'freq')




1
2
3


In [42]:
a1.to_csv(fnames['small_right'])
a2.to_csv(fnames['large_right'])
b1.to_csv(fnames['small_left'])
b2.to_csv(fnames['large_left'])